In [9]:
import pandas as pd

In [10]:
data = pd.read_csv("output.csv", sep=",")
data = data.drop(columns=['date', 'likes'])
data.head()

,username,title,start,end,com
0,Robert S,"Flying the Unfriendly Skies of Air France, and...",Denver,"- Stockholm, International]",Where to begin!!!! Denver to Paris flight on A...
1,Sophia T,Missed connection because of the airline’s fau...,Washington DC,"- Porto, International]",My first flight from DC to Paris got delayed b...
2,Khaoula M,Fantastic Service and Entertainment on My Long...,Tunis,"- Miami, International]",I recently flew with Air France from Tunisia t...
3,Dougie,Disorganised airline],Edinburgh,"- Valletta, Europe]","Rubbish organisation, missed connection due to..."
4,Arash C,Air France Business crew are very rude to othe...,Paris,"- Miami, International]",Rude Cabin Chef AF092.Rude Cabin Chef and crew...


In [13]:
print(f"Taille de data : {len(data)}")
data = data.dropna(how='any')
print(f"Taille de data : {len(data)}")

Taille de data : 12910
Taille de data : 12895


In [14]:
data['end_'] = data['end'].apply(lambda x: x.split(',')[0])
data = data.drop(columns=['date', 'likes'])
data.head()

,username,title,start,end,com,end_
0,Robert S,"Flying the Unfriendly Skies of Air France, and...",Denver,"- Stockholm, International]",Where to begin!!!! Denver to Paris flight on A...,- Stockholm
1,Sophia T,Missed connection because of the airline’s fau...,Washington DC,"- Porto, International]",My first flight from DC to Paris got delayed b...,- Porto
2,Khaoula M,Fantastic Service and Entertainment on My Long...,Tunis,"- Miami, International]",I recently flew with Air France from Tunisia t...,- Miami
3,Dougie,Disorganised airline],Edinburgh,"- Valletta, Europe]","Rubbish organisation, missed connection due to...",- Valletta
4,Arash C,Air France Business crew are very rude to othe...,Paris,"- Miami, International]",Rude Cabin Chef AF092.Rude Cabin Chef and crew...,- Miami


In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12910 entries, 0 to 12909
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   username  12910 non-null  object
 1   title     12909 non-null  object
 2   start     12896 non-null  object
 3   end       12896 non-null  object
 4   com       12910 non-null  object
dtypes: object(5)
memory usage: 504.4+ KB
